In [1]:
import sys
import os
from pathlib import Path
sys.path.append(Path.cwd().parent.as_posix())
from util import *
from visualizer import visualizer
vis = visualizer()

In [2]:
def get_prim_and_conv(structure):
    sga = SpacegroupAnalyzer(structure)
    prim = sga.get_primitive_standard_structure()
    conv = sga.get_conventional_standard_structure()
    return prim, conv

In [3]:
structures = test_structures()

  0%|          | 0/100 [00:00<?, ?it/s]

[I] Loaded 100 structures


In [7]:
st = structures[2]

In [8]:
prim, conv = get_prim_and_conv(st)

In [10]:
map_sites_conv_to_prim(conv, prim)

([Element Na,
  Element Na,
  Element Ca,
  Element Ca,
  Element Ta,
  Element Ta,
  Element Ti,
  Element Ti,
  Element O,
  Element O,
  Element O,
  Element O,
  Element O,
  Element O,
  Element O,
  Element O,
  Element O,
  Element O,
  Element O,
  Element O],
 [array([0.754485, 0.22613 , 0.750071]),
  array([0.754485, 0.77387 , 0.250071]),
  array([0.237176, 0.704963, 0.749993]),
  array([0.237176, 0.295037, 0.249993]),
  array([4.94252e-01, 2.43969e-01, 2.24000e-04]),
  array([0.494252, 0.756031, 0.500224]),
  array([0.996668, 0.244979, 0.503044]),
  array([0.996668, 0.755021, 0.003044]),
  array([0.322843, 0.732302, 0.248053]),
  array([0.761005, 0.46989 , 0.537855]),
  array([0.180888, 0.469022, 0.96101 ]),
  array([0.837118, 0.962252, 0.54319 ]),
  array([0.247478, 0.95731 , 0.955643]),
  array([0.668083, 0.23327 , 0.250915]),
  array([0.322843, 0.267698, 0.748053]),
  array([0.761005, 0.53011 , 0.037855]),
  array([0.180888, 0.530978, 0.46101 ]),
  array([0.837118, 0.0377

In [11]:
prim

Structure Summary
Lattice
    abc : 5.456484 5.550569 9.475464738856905
 angles : 90.0 125.08859689766035 90.0
 volume : 234.8250537411465
      A : 0.0 5.456484 0.0
      B : 5.550569 0.0 0.0
      C : 0.0 -5.446898997010619 -7.75343300310806
    pbc : True True True
PeriodicSite: Na (1.255, 0.03127, -5.816) [0.7545, 0.2261, 0.7501]
PeriodicSite: Na (4.295, 2.755, -1.939) [0.7545, 0.7739, 0.2501]
PeriodicSite: Ca (3.913, -2.791, -5.815) [0.2372, 0.705, 0.75]
PeriodicSite: Ca (1.638, -0.06754, -1.938) [0.2372, 0.295, 0.25]
PeriodicSite: Ta (1.354, 2.696, -0.001737) [0.4943, 0.244, 0.000224]
PeriodicSite: Ta (4.196, -0.02779, -3.878) [0.4943, 0.756, 0.5002]
PeriodicSite: Ti (1.36, 2.698, -3.9) [0.9967, 0.245, 0.503]
PeriodicSite: Ti (4.191, 5.422, -0.0236) [0.9967, 0.755, 0.003044]
PeriodicSite: O (4.065, 0.4105, -1.923) [0.3228, 0.7323, 0.2481]
PeriodicSite: O (2.608, 1.223, -4.17) [0.761, 0.4699, 0.5379]
PeriodicSite: O (2.603, -4.248, -7.451) [0.1809, 0.469, 0.961]
PeriodicSite: O (5

In [58]:
results = []
for st in structures:
    prim, conv = get_prim_and_conv(st)
    prim_lattice_inv = prim.lattice.inv_matrix
    new_fracs = []
    new_species = []
    for site in conv:
        new_frac = np.dot(site.coords, prim_lattice_inv)
        # if not any(partial(pbc_diff,new_frac) == partial(f) for f in new_fracs):
        #     new_species.append(site.specie)
        if not any(map(partial(is_periodic_image, new_frac), new_fracs)):
            new_fracs.append(new_frac % 1)
            new_species.append(site.specie)
        new_fracs = [new_frac % 1 for new_frac in new_fracs]
    results.append(np.allclose(np.array(new_fracs),prim.frac_coords % 1))

In [63]:
rand_strctures = []
for st in structures:
    rand_strctures.append(test_space_group_projection(st)["proj_structure"])

In [75]:
prim, conv = get_prim_and_conv(rand_strctures[0])

In [77]:
cif, sym_info, num_sites, formula = process_conventional_with_sym(prim)

In [79]:
prim.num_sites

4

In [80]:
prim

Structure Summary
Lattice
    abc : 5.680331055637312 5.680331055637312 5.680331055637313
 angles : 59.99999999999999 59.99999999999999 59.99999999999999
 volume : 129.60028152198547
      A : 6.459186413757549e-16 4.016600608825684 4.016600608825684
      B : 4.016600608825684 0.0 4.016600608825684
      C : 4.0166006088256845 4.016600608825684 4.918917079051683e-16
    pbc : True True True
PeriodicSite: Tm (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
PeriodicSite: Mg (4.017, 4.017, 4.017) [0.5, 0.5, 0.5]
PeriodicSite: Hg (2.008, 2.008, 2.008) [0.25, 0.25, 0.25]
PeriodicSite: Hg (6.025, 6.025, 6.025) [0.75, 0.75, 0.75]

In [90]:
Structure.from_str(cif, fmt="cif").cart_coords[np.unique(sym_info["anchors"])]

/home/holywater2/crystal_gen/mattergen/.venv/lib/python3.10/site-packages/pymatgen/io/cif.py:1304: UserWarning: Issues encountered while parsing CIF: Skipping relative stoichiometry check because CIF does not contain formula keys.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


array([[0.        , 0.        , 0.        ],
       [4.0166005 , 4.0166005 , 4.0166005 ],
       [2.00830025, 2.00830025, 2.00830025]])

In [ ]:
for st in structures:
    prim, conv = get_prim_and_conv(st)
    
    cif, sym_info, num_sites, formula = process_conventional_with_sym(prim)   
    psites = project_sites()
    print(prim.num_sites, conv.num_sites)

In [92]:
prim.frac_coords

array([[0.  , 0.  , 0.  ],
       [0.5 , 0.5 , 0.5 ],
       [0.25, 0.25, 0.25],
       [0.75, 0.75, 0.75]])